# How many signal events do we need?

This notebook studies the SIC achieved by boosted decision trees depending on the number of signal events present in the data.

In [ ]:
import numpy as np
import subprocess
import sys
import tqdm
from os.path import dirname, join, realpath
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

# adding parent directory to path
parent_dir = dirname(realpath(globals()["_dh"][0]))
sys.path.append(parent_dir)

import sk_cathode
from sk_cathode.classifier_models.boosted_decision_tree import HGBClassifier
from sk_cathode.utils.ensembling_utils import EnsembleModel

The same input data as in `demos/weak_supervision.ipynb` are used here. We make use of the same separation into train/validation/test data. However, we don't make use of the extra train/validation signal, as we won't train a supervised classifier.

In [ ]:
data_path = "./input_data/"

In [ ]:
# data preparation (download and high-level preprocessing)
if not (Path(data_path) / "innerdata_test.npy").exists():
    data_preparation = Path(sk_cathode.__file__).parents[1] / 'demos' / 'utils' / 'data_preparation.py'
    process = subprocess.run(f"{sys.executable} {data_preparation} --outdir {data_path}", shell=True, check=True)

In [ ]:
# data loading
innerdata_train = np.load(join(data_path, "innerdata_train.npy"))
innerdata_val = np.load(join(data_path, "innerdata_val.npy"))
innerdata_test = np.load(join(data_path, "innerdata_test.npy"))
innerdata_extrabkg_train = np.load(join(data_path, "innerdata_extrabkg_train.npy"))
innerdata_extrabkg_val = np.load(join(data_path, "innerdata_extrabkg_val.npy"))
innerdata_extrabkg_test = np.load(join(data_path, "innerdata_extrabkg_test.npy"))
innerdata_extrasig = np.load(join(data_path, "innerdata_extrasig.npy"))

# Enriching the test set with extra signal.
# We could use all, but this way it's consistent with previous notebooks.
innerdata_extrasig_test = innerdata_extrasig[:20000]

In [ ]:
# mix together train and validation set for different splits
innerdata_train_val = np.vstack([innerdata_train, innerdata_val])
innerdata_extrabkg_train_val = np.vstack([innerdata_extrabkg_train, innerdata_extrabkg_val])

## Pick and mix signal events

Here we pick how many signal events we want to use. The notebook deals with a single number of events at a time.

In [ ]:
signal_events = 125

indices = np.where(innerdata_train_val[:, -1] == 1)[0]
print('Available signal events:', len(indices))

rng = np.random.default_rng(seed=42)  # For reproducibility
rng.shuffle(indices)

# Keep only the requested number of signal events
selection = np.ones(len(innerdata_train_val), dtype=bool)
selection[indices[signal_events:]] = False
innerdata_train_val_nsignals = innerdata_train_val[selection]

## Tree definitions

We define functions for the BDT-based classification:
* `train()` is used to construct the model. It operates on two datasets: the "data", which contains a small fraction of signal events, and the "background" which contains none.
* `get_selection_indices()` is used to select the anomalous items using the fitted model. The dataset passed to this function contains a lot more signal than in `train()`, so the model must not be changed here.

In [ ]:
def train(data: np.array, background: np.array, save_label: str,
          n_classifiers:int=10, split_key:int=1337):
    """
    Trains an ensemble of trees with different train/val splits.
    
    Parameters
    ----------
    data : np.array
        Array containing "data" features, i.e. a mix of background and signal.
    background : np.array
        Array containing "background" features.
    save_label : str
        A string inserted in the path when saving the trained models.
    n_classifier : int
        The number of trees to ensemble.
    split_key : int
        Used in the initialization of the train/val split. Should be
        significantly larger than n_classifiers.

    Returns
    -------
    An ensemble model that can be used to obtain the averaged score for events
    in the test set.
    """

    model_list = []
    for i in range(n_classifiers):

        # different split per classifier
        # (could also do this more controlled via a fixed k-folding scheme)
        innerdata_train, innerdata_val = train_test_split(
            data, train_size=0.8, random_state=split_key+i)
        innerdata_extrabkg_train, innerdata_extrabkg_val = train_test_split(
            background, train_size=0.8, random_state=split_key+i)

        # assigning label 1 to "data"
        clsf_train_data = innerdata_train
        clsf_train_data[:, -1] = np.ones_like(clsf_train_data[:, -1])
        clsf_val_data = innerdata_val
        clsf_val_data[:, -1] = np.ones_like(clsf_val_data[:, -1])

        # and label 0 to background
        clsf_train_bkg = innerdata_extrabkg_train
        clsf_train_bkg[:, -1] = np.zeros_like(clsf_train_bkg[:, -1])
        clsf_val_bkg = innerdata_extrabkg_val
        clsf_val_bkg[:, -1] = np.zeros_like(clsf_val_bkg[:, -1])

        # mixing together and shuffling
        clsf_train_set = np.vstack([clsf_train_data, clsf_train_bkg])
        clsf_val_set = np.vstack([clsf_val_data, clsf_val_bkg])
        clsf_train_set = shuffle(clsf_train_set, random_state=42)
        clsf_val_set = shuffle(clsf_val_set, random_state=42)

        # fit scaler
        scaler = StandardScaler()
        scaler.fit(clsf_train_set[:, 1:-1])

        # train classifier
        classifier_savedir = f"./trained_classifier_idealized-ad_ensemble{save_label}/model_{i}/"
        classifier = HGBClassifier(save_path=classifier_savedir,
                                   early_stopping=True, max_iters=None,
                                   verbose=False)

        # We don't want to overwrite the model if it already exists.
        if not (Path(classifier_savedir) / "CLSF_models").exists():
            X_train = scaler.transform(clsf_train_set[:, 1:-1])
            y_train = clsf_train_set[:, -1]
            X_val = scaler.transform(clsf_val_set[:, 1:-1])
            y_val = clsf_val_set[:, -1]
            classifier.fit(X_train, y_train, X_val, y_val)
        else:
            print(f"The model exists already in {classifier_savedir}. Remove first if you want to overwrite. Loading its best state now.")
            classifier.load_best_model()

        # merge scaler and classifier into a single pipeline model
        pipeline = make_pipeline(scaler, classifier)
        model_list.append(pipeline)

    # Now merging all these models into a single ensemble model
    return EnsembleModel(model_list)

In [ ]:
def get_selection_indices(model: EnsembleModel, data: np.array, truth: np.array, efficiency=1e-4):
    """
    Returns a boolean array with the indices of "anomalous" rows.
    
    We select the rows with the highest prediction. The threshold is set
    according to the *background* selection efficiency (assuming this can be
    obtained from sidebands).
    
    Paramters
    ---------
    model : EnsembleModel
        The model to use for the selection.
    data : np.array
        The features to use to perform the selection.
    truth : np.array
        Truth labels so we can compute the background efficiency.
    efficiency : np.array
        The background selection efficiency.
    """

    predictions = model.predict(data).flatten()
    threshold = np.quantile(predictions[~truth.astype(bool)], 1 - efficiency)
    return predictions > threshold

## Train models

We train 10 classifiers with different initializations so we can check the variance of the algorithm:

In [ ]:
classifiers = [
    train(innerdata_train_val_nsignals, innerdata_extrabkg_train_val, f'-{signal_events}_{i}', split_key=(i + 1) * 1337)
    for i in tqdm.tqdm(range(10))
]

# Evaluation

We call `get_selection_indices` for each trained model and compute the SIC, then report the average and standard deviation.

In [ ]:
# now let's evaluate the signal extraction performance on the same test set

clsf_test_set = np.vstack([innerdata_test,
                           innerdata_extrabkg_test,
                           innerdata_extrasig_test])

X_test = clsf_test_set[:, 1:-1]
y_test = clsf_test_set[:, -1]

sics = []
for ensemble in classifiers:
    selection = get_selection_indices(ensemble, X_test, y_test, efficiency=1e-4)
    tpr = (y_test[selection]).sum() / y_test.sum()
    fpr = (1 - y_test[selection]).sum() / len(y_test)
    sics.append(tpr / np.sqrt(fpr))

print(f'The SIC we get is: {np.mean(sics):.1f} +/- {np.std(sics):.1f}')

Here are the results for varying numbers of signal events used in the training:

| <div style="min-width:100px">Signal events | <div style="min-width:100px">$S/\sqrt B$ | <div style="min-width:100px">SIC $\pm$ std |
|:-------------:|:-----------:|:------------:|
| 525 (all)     | 1.8         | $13.1\pm0.4$ |
| 200           | 0.7         | $9.4\pm0.5$  |
| 150           | 0.5         | $4.3\pm2.4$  |
| 125           | 0.44        | $2.2\pm2.3$  |
| 100           | 0.35        | $0.4\pm0.9$  |

In [ ]:
print(len(innerdata_train_val_nsignals), len(innerdata_extrabkg_train_val))